In [ ]:
"""
Import the necessary libraries here
"""
import math
import subprocess
import pdb
import pickle
import os
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from utils import *
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,3)

In [ ]:
"""
Define the haversine distance function
"""
def haversine_distance(point1, point2):
    """
    Computes the haversie distance (in meters) between 
    two points in the global coordinate systems.
    Source: http://www.movable-type.co.uk/scripts/latlong.html
    point1 - tuple (longitude, latitude) in degrees
    point2 - tuple (longitude, latitude) in degrees
    """
    lat1 = math.radians(point1[0])
    lat2 = math.radians(point2[0])
    delta_lat  = point1[0] - point2[0]
    delta_long = point1[1] - point2[1]

    R = 6371000.0 # In meters
    a = math.sin(math.radians(delta_lat)/2)**2 + math.cos(lat1)*math.cos(lat2)*(math.sin(math.radians(delta_long)/2)**2)
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R*c

    return d

In [ ]:
def angle_2points(point1, point2):
    """
    point1 - numpy ndarray
    point2 - numpy ndarray
    Ref: https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python/13849249
    
    Returns angle between point1 and point2 in degrees
    """
    
    c = np.dot(point1, point2)
    if c == 0:
        return 90.00
    else:
        c /= (np.linalg.norm(point1) * np.linalg.norm(point2))
    return np.degrees(np.arccos(np.clip(c, -1, 1)))

def baseline_angle_1(point1, point2, center):
    """
    point1 - (latitude/phi, longitude/theta, height) triplet
    point2 - (latitude/phi, longitude/theta, height) triplet
    height - (latitude/phi, longitude/theta, height) triplet
    Formula obtained by converting from global coordinates to x,y,z coordinate
    with x, y axes on the equator plane and z from south pole to north pole
    
    Returns baseline angle between viewpoints 1 and 2
    """
    phi1 = math.radians(point1[0])
    theta1 = math.radians(point1[1])
    h1 = point1[2]
    
    phi2 = math.radians(point2[0])
    theta2 = math.radians(point2[1])
    h2 = point2[2]
    
    phic = math.radians(center[0])
    thetac = math.radians(center[1])
    hc = center[2]
    
    R = 6371000.0 # In meters
    
    """
    Convert to x, y, z coordinates
    """
    x1 = (R + h1)*math.cos(phi1)*math.sin(theta1)
    y1 = -(R + h1)*math.cos(phi1)*math.cos(theta1)
    z1 = (R+h1)*math.sin(phi1)
    X1 = np.array([x1, y1, z1])
    
    x2 = (R + h2)*math.cos(phi2)*math.sin(theta2)
    y2 = -(R + h2)*math.cos(phi2)*math.cos(theta2)
    z2 = (R+h2)*math.sin(phi2)
    X2 = np.array([x2, y2, z2])
    
    xc = (R + hc)*math.cos(phic)*math.sin(thetac)
    yc = -(R + hc)*math.cos(phic)*math.cos(thetac)
    zc = (R+hc)*math.sin(phic)
    Xc = np.array([xc, yc, zc])
    
    return angle_2points(X1-Xc, X2-Xc)

def baseline_angle_2(point1, point2, center):    
    
    """
    point1 - (latitude/phi, longitude/theta, height) triplet
    point2 - (latitude/phi, longitude/theta, height) triplet
    height - (latitude/phi, longitude/theta, height) triplet
    Formula obtained by computing pairwise haversine distance
    and using cosine formula for triangles
    
    Returns baseline angle between viewpoints 1 and 2
    """
    
    d1 = haversine_distance(point1[:2], center[:2])
    d2 = haversine_distance(point2[:2], center[:2])
    d3 = haversine_distance(point1[:2], point2[:2])
    
    cos_beta = (d1*d1 + d2*d2 - d3*d3)/(2*d1*d2)
    
    return math.degrees(math.acos(cos_beta))
    

In [ ]:
"""
Define the base directory and read in the image
and text files
"""
base_dir = 'dataset/test/0096/'
files = subprocess.check_output(['ls', base_dir]).split()
txtfiles = []
imgfiles = []
for f in files:
    if f[-3:] == 'txt':
        txtfiles.append(f)
    else:
        imgfiles.append(f)
        
print("Number of images read: %d"%(len(txtfiles)))

In [ ]:
"""
Create the dictonary of target points
"""
targets = create_target_cache('dataset/test/', '0096/')
targetIDs = targets.keys()


In [ ]:
""" 
Visualizing the aligned patches for each target

"""

fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 30)

for targetIDX in range(100, len(targets.keys())):
    targetID = targets.keys()[targetIDX]
    target = targ
    
    
    ets[targetID]
    #print(target)
    nViews = len(target['views'])
    if nViews == 1:
        print('Only 1 view! Skipping')
        continue
        
    print('Number of views: %d'%(nViews))
    
    viewsList = []
    
    for i in range(nViews):
        try:
            imgCurr = Image.open('dataset/test/' + target['views'][i]['imagePath'])
        except IOError: 
            continue
        drawCurr = ImageDraw.Draw(imgCurr)
        alignDataCurr = target['views'][i]['alignData']
        width, height = imgCurr.size
        
        l = int(float(alignDataCurr[1]) - 96)
        t = int(float(alignDataCurr[2]) - 96)
        r = l + 192 - 1
        b = t + 192 - 1
        #l = int(float(alignDataCurr[3]))
        #t = int(float(alignDataCurr[36]))
        #r = int(float(alignDataCurr[41]))
        #b = int(float(alignDataCurr[42]))
        
        box = [(l, t), (r, t), (r, b), (l, b), (l, t)]
        #drawCurr.ellipse(box, fill=(255, 0, 0, 128))
        drawCurr.polygon(box, outline=(0, 0, 255))
        drawCurr.text((10, 10), 'View: %d , SSI score: %.3f'%(i, float(alignDataCurr[32])), font = fnt, fill=(255, 128, 0, 255))
        #drawCurr.text((10, 50), 'Total Energy, %d, Inlier Ratio: %.3f'%(int(float(alignDataCurr[30])), float(alignDataCurr[31])), font=fnt, fill=(255, 128, 255, 255))
        viewsList.append(imgCurr.resize((int(width/1.5), int(height/1.5))))
        print('View: %d , SSI score: %s , Total Energy, %s, Inlier Ratio: %s'%(i, alignDataCurr[32], alignDataCurr[30], alignDataCurr[31]))
        
    widths, heights = zip(*(ig.size for ig in viewsList))
    total_width = sum(widths)
    max_height = max(heights)
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in viewsList:
        new_im.paste(im, (x_offset, 0))
        x_offset += im.size[0]
        
    
    plt.imshow(np.asarray(new_im))
    plt.show()
    pdb.set_trace()


In [ ]:
"""
Group the matches into categories of 
 (0) beta < 30
 (1) beta < 60
 (2) beta < 90
 (3) beta < 120
 (4) all other beta 
"""

def category_assignment(beta):
    category = 0
    if beta < 30:
        category = 0
    elif beta < 60:
        category = 1
    elif beta < 90:
        category = 2
    elif beta < 120:
        category = 3
    else:
        category = 4
    return category

In [ ]:
print("Number of targets: %d"%(len(targetIDs)))

In [ ]:
"""
Divide the matching views based on baseline angle
"""
categories = [[], [], [], [], []]

for i in range(len(targetIDs)):
    targetIDCurr = targetIDs[i]
    targetCurr = targets[targetIDCurr]
    #print('\n================================================================\n')
    #print('Number of views for target %d: %d'%(targetIDCurr, len(targetCurr['views'])))
    for j in range(len(targetCurr['views'])-1):
        for k in range(j+1, len(targetCurr['views'])):
            b_angle = baseline_angle_1(targetCurr['views'][j]['cameraCoord'], targetCurr['views'][k]['cameraCoord'], targetCurr['targetCoord'])
            align_data_j = targetCurr['views'][j]['alignData']
            align_data_k = targetCurr['views'][k]['alignData']
            if float(align_data_j[32]) >= 0.19 and float(align_data_k[32]) >= 0.19:
            #print('B angle: %.4f'%(b_angle))
                print('Img 1: %s ::: Img 2: %s'%(targetCurr['views'][j]['imagePath'], targetCurr['views'][k]['imagePath']))
                categories[category_assignment(b_angle)].append([targetCurr['views'][j]['imagePath'], targetCurr['views'][k]['imagePath'], b_angle, align_data_j, align_data_k])


In [ ]:
for i, category in enumerate(categories):
    print('Number of points in category # %d: %d'%(i, len(category)))

In [ ]:
for i in range(len(categories)):
    print('=====================================================================')
    print('Category # %d'%(i))
    for j in range(len(categories[i])):
        try:
            images = map(Image.open, ('dataset/test/'+name for name in categories[i][j][0:2]))
        except IOError:
            continue
            
        image_1_align = categories[i][j][3]
        image_2_align = categories[i][j][4]
        
        
        # Obtain a 192x192 patch centered at the aligned center
        l1 = int(float(image_1_align[1]) - 96)
        t1 = int(float(image_1_align[2]) - 96)
        r1 = l1 + 192 - 1
        b1 = t1 + 192 - 1
        box1 = (l1, t1, r1, b1)
        
        l2 = int(float(image_2_align[1]) - 96)
        t2 = int(float(image_2_align[2]) - 96)
        r2 = l2 + 192 - 1
        b2 = t2 + 192 - 1
        box2 = (l2, t2, r2, b2)
        
        """
        draw1 = ImageDraw.Draw(images[0])
        draw2 = ImageDraw.Draw(images[1])
        
        l1 = int(float(image_1_align[1]) - 5)
        t1 = int(float(image_1_align[2]) - 5)
        r1 = l1 + 10 - 1
        b1 = t1 + 10 - 1
        box1 = [(l1, t1), (r1, b1)]
        
        l2 = int(float(image_2_align[1]) - 5)
        t2 = int(float(image_2_align[2]) - 5)
        r2 = l2 + 10 - 1
        b2 = t2 + 10 - 1
        box2 = [(l2, t2), (r2, b2)]
        
        draw1.ellipse(box1, fill=(255,255,255,128))
        draw2.ellipse(box2, fill=(255,255,255,128))
        """
        images[0] = images[0].crop(box1)
        images[1] = images[1].crop(box2)
        
        widths, heights = zip(*(i.size for i in images))
        total_width = sum(widths)
        max_height = max(heights)
        new_im = Image.new('RGB', (total_width, max_height))

        x_offset = 0
        for im in images:
            new_im.paste(im, (x_offset, 0))
            x_offset += im.size[0]
            
        #new_im.save('/tmp/scratch_mount/Fall2017-CS381V/project/3d-generic/dataset/saved_images/category%d/%s_%s_%.3f.jpg'%(i, categories[i][j][0][5:], categories[i][j][1][5:],categories[i][j][2]))
        plt.imshow(np.asarray(new_im))
        plt.show()
        pdb.set_trace()